ACTUALIZA LAS PROPIEDADES QUE TIENEN MIDIFICACIONES

In [1]:
fef

NameError: name 'fef' is not defined

In [2]:
import ee
ee.Authenticate()

True

In [3]:
import ee
import geemap
# metodo de clasificacion
import jenkspy

import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Polygon

import pandas as pd
import numpy as np
import ipywidgets as widgets

import openpyxl

In [4]:
ee.Initialize()

In [5]:
ruta_catastro='projects/ee-bismarksr17/assets/LOTES_CANHA_2025'
ruta_ndvi = 'projects/ee-bismarksr17/assets/NDVI_02_2025'

CODIGO_PROPIEDAD='unidad_01'
NOM_PROPIEDAD='unidad_02'
CODIGO_CANHERO='unidad_03'
NOM_CANHERO='unidad_04'

VARIEDAD='variedad'
ESTADO='RENOVACION'
SUPERFICIE='area'

cods_error = []

In [6]:
def asig_cat(label):
    if label == 1:
        return 0
    else:
        return label * 10

vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# parametro de visualizacion LOTES RENOVACION
vis_params_renov = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

In [7]:
# importa codigos de propiedades para descartar
path_cat_descartar = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\ESTIMATIVAS\V06\SHP\LOTES_CANHA_2025_DESCARTAR.shp'
gdf_cat_descartar = gpd.read_file(path_cat_descartar)
lista_props_descartar = list(set(gdf_cat_descartar['unidad_01']))
lista_props_descartar = [int(i) for i in lista_props_descartar]
print(len(lista_props_descartar))
print(lista_props_descartar)

74
[2177, 1410, 2185, 14, 1423, 658, 1173, 1174, 1175, 1944, 2073, 1953, 1442, 673, 2212, 1830, 1063, 1064, 1065, 1831, 1067, 1707, 172, 2086, 301, 302, 303, 304, 305, 308, 2101, 429, 1591, 2231, 1977, 818, 1723, 823, 824, 1604, 711, 459, 587, 589, 1615, 1871, 2000, 1106, 1618, 2129, 80, 342, 2008, 858, 1500, 1756, 2270, 1507, 1895, 1769, 2282, 2283, 108, 2156, 110, 2284, 368, 1521, 1650, 2163, 1654, 374, 1148, 1150]


In [8]:
# importa todos los codigos de propiedad
path_cat = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\ESTIMATIVAS\V06\SHP\LOTES_CANHA_2025.shp'
gdf_cat = gpd.read_file(path_cat)
lista_props = list(set(gdf_cat['unidad_01']))
len(lista_props)

1641

In [9]:
lista_props_final = [x for x in lista_props if x not in lista_props_descartar]
len(lista_props_final)
print(lista_props_final)

[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 54, 55, 57, 61, 62, 63, 66, 67, 68, 71, 72, 74, 76, 78, 79, 83, 84, 85, 89, 92, 93, 94, 98, 100, 104, 105, 106, 111, 112, 113, 114, 116, 117, 118, 119, 122, 123, 124, 125, 126, 127, 129, 132, 133, 135, 136, 137, 139, 140, 141, 142, 145, 147, 149, 150, 155, 159, 161, 162, 164, 168, 171, 173, 174, 176, 178, 179, 182, 183, 184, 185, 186, 187, 188, 189, 195, 197, 198, 201, 205, 206, 209, 211, 212, 213, 216, 217, 218, 219, 220, 221, 222, 225, 226, 227, 228, 229, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 246, 247, 249, 250, 251, 253, 256, 257, 258, 259, 260, 261, 263, 264, 265, 267, 268, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 298, 299, 300, 306, 307, 309, 310, 311, 313, 314, 320, 324, 325, 326, 327, 328, 329, 330, 331, 33

In [25]:
lista_props_final = [30, 1, 19, 47]
len(lista_props_final)

4

In [26]:
cods_error

[]

In [27]:
PATH_INTERSECT_BASE = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\ESTIMATIVAS\V06\SHP\INTERSECT_0.shp'
df_intersects = gpd.read_file(PATH_INTERSECT_BASE)
df_intersects.shape

(0, 27)

In [31]:
estado_proceso = widgets.Output(layout={'border': '1px solid black'})
estado_proceso

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [32]:
contador = 0
for cod_prop in lista_props_final:
    propiedad = ee.FeatureCollection(ruta_catastro)\
                .filter(ee.Filter.eq(CODIGO_PROPIEDAD, cod_prop))
    lotes_canha = propiedad.filter(ee.Filter.neq(VARIEDAD, ESTADO))
    lotes_renov = propiedad.filter(ee.Filter.eq(VARIEDAD, ESTADO))
    NDVI = ee.Image(ruta_ndvi)
    NDVI_clip = NDVI.clip(lotes_canha.geometry())
    # crea un sample de los valore de pixel del NDVI
    NDVI_values = NDVI_clip.sampleRegions(lotes_canha.geometry())
    # reduce el resultado a valores de NDVI
    pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['b1']).get('list').getInfo()
    # aplica metodo Jenks
    breaks = jenkspy.jenks_breaks(pixel_values, n_classes=8)
    NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)
    NDVI_class_clip = NDVI_class.clip(lotes_canha.geometry())
    clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)
    vector = clasify.reduceToVectors(**{
        'geometry': lotes_canha.geometry(),
        'crs': clasify.projection(),
        'scale': 10,
        'geometryType': 'polygon',
        'eightConnected': False
    })
    
    DF_VECTOR = vector
    
    lotes_local = geemap.ee_to_gdf(lotes_canha)
    
    lista = vector.toList(vector.size()).getInfo()
    lista_vector=[]
    for item in lista:
        dic = {'geometry':Polygon(item['geometry']['coordinates'][0]), 'count':item['properties']['count'], 'label':item['properties']['label']}
        lista_vector.append(dic)
    
    vector_local = gpd.GeoDataFrame(lista_vector)
    vector_local.set_crs("EPSG:32720", inplace=True)  # ajusta si usas otra zona UTM
    
    try:
        #lotes_local = lotes_local.to_crs("EPSG:32720")
        intersect = overlay(lotes_local, vector_local, how="intersection")
    except TypeError as e:
        cods_error.append(cod_prop)
        continue

    intersect.crs = "EPSG:4326"
    intersect = intersect.to_crs(epsg=32720)
    intersect['area_2'] = intersect['geometry'].area/10000
    
    area_01 = intersect['area_2'].sum()
    area_02 = lotes_local[SUPERFICIE].sum()
    
    if len(intersect) > 0:
        area_diff = (area_02 - area_01) / len(intersect)
    else:
        area_diff = 0  # o np.nan, dependiendo de cómo lo quieras manejar
    
    intersect['area_2'] = intersect['area_2'] + area_diff
    
    DF = intersect.copy()
    dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc='sum')
    dina2 = dina.reset_index()
    
    dina2['tch'] = dina2['label'].apply(asig_cat)
    
    # Cargar el archivo de Excel existente
    wb = openpyxl.load_workbook('DATA_EST.xlsx')
    # Seleccionar la hoja de trabajo
    ws = wb['data']
    
    for i in range(0, len(dina2)):
        new_row = [dina2.iloc[i,0], dina2.iloc[i,1], dina2.iloc[i,2], dina2.iloc[i,3], dina2.iloc[i,4], dina2.iloc[i,5], dina2.iloc[i,6]]
        ws.append(new_row)
    wb.save('DATA_EST.xlsx')
    
    df_intersects = pd.concat([df_intersects, DF], ignore_index=True)
    
    contador = contador + 1 
    with estado_proceso:
        estado_proceso.clear_output()
        display(f'ESTADO...')
        display(f'Se proceso propiedad: {cod_prop}')
        display(f'Contador: {contador} de {len(lista_props_final)}')
print('FIN......!')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13628\4225511096.py:48: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:32720

  intersect = overlay(lotes_local, vector_local, how="intersection")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13628\4225511096.py:48: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:32720

  intersect = overlay(lotes_local, vector_local, how="intersection")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13628\4225511096.py:48: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326


FIN......!


In [ ]:
cods_error

In [ ]:
df_intersects.tail(5)

In [ ]:
len(df_intersects)

In [ ]:
df_intersects.to_file("INTERSECT_01.shp", driver="ESRI Shapefile")

In [33]:
df_intersects['area_2'].sum()

2826.285151940068

In [ ]:
df_intersects.head(3)